In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from netgen.csg import *
from ngsolve import *
from ngsolve.webgui import Draw
# from netgen import gui
from ngsolve.comp import ConvertOperator
import numpy as np

import time

Geometrie: Spule mit Luftspalt und Stahlplatte dazwischen

In [ ]:
geo = CSGeometry()
box_size = 2
box = OrthoBrick(Pnt(-box_size,-box_size,-box_size), Pnt(box_size,box_size,box_size)).bc("outer")

lx = 0.4
lz = 0.5
d = 0.1
lp = 0.3
core = OrthoBrick(Pnt(-lx,-d/2,-lz),Pnt(lx,d/2,lz))- \
           OrthoBrick(Pnt(d-lx,-d,-lz+d),Pnt(lx-d,d,lz-d))- \
           OrthoBrick(Pnt(d,-d,-d/2),Pnt(lx+d,d,d/2)).maxh(0.2).mat("core")

plate = OrthoBrick(Pnt(d/2,-lp,-d/4),Pnt(2*lx-d,lp,d/4)).mat('plate')


coil = (Cylinder(Pnt(-lx+d/2,0,0), Pnt(-lx+d/2,0,1), 3*d) - \
        Cylinder(Pnt(-lx+d/2,0,0), Pnt(-lx+d/2,0,1), d)) * \
        OrthoBrick (Pnt(-1,-1,-lz+2*d),Pnt(1,1,lz-2*d)).maxh(0.2).mat('coil')

air = (box - coil - plate -core).mat('air') 

geo.Add(plate)
geo.Add(core)
geo.Add(coil)
geo.Add(air, transparent=True)

mesh = Mesh(geo.GenerateMesh(maxh=0.5))
mesh.Curve(1)
Draw (mesh)

$$
\DeclareMathOperator{\Grad}{grad}
\DeclareMathOperator{\Curl}{curl}
\DeclareMathOperator{\Div}{div}
\DeclareMathOperator{\R}{\mathbb{R}}
\DeclareMathOperator{\real}{real}
\DeclareMathOperator{\reg}{reg}
$$
### Magnetostatisches Problem ohne Stahlplatte

Finde $ u \in H(\Curl)$ sodass 
$$
\int \mu^{-1} \Curl u \Curl v + \reg \int  u  v= \int j v \qquad  \forall \, v \in H(\Curl) \tag{1}
$$

Wir nehmen zunächst an, dass der `core` aus einem tollen Material besteht das immer eine hohe, lineare Permeabilität hat. Um in den nichtlinearen Bereich des Stahls zu kommen wollen wir ein Magnetfeld zwischen 1.0 und 1.5 Tesla im Lufspalt. 

In folgender Rechnung wird vernachlässigt, dass der Luftspalt (bzw dann die zwei Luftspalte) kleiner ist wenn sich eine Stahlplatte darin befindet, daher ist das Ziel ohne Stahlplatte auf ca 1T zu kommen. Außerdem wird angenommen dass es keine Streuflüsse gibt. 

Es gilt für das Magnetfeld im Luftspalt $B_l$ gilt $B = \frac{\Phi}{A}$ Wobei $\Phi$ die magnetische Flussdichte und $A$ die Querschnittsfläche des Luftspalts ist. Die von der Spule erzeugte Durchflutung $\Theta$ beträgt $4d(l_z-d)*j$, falls die Stromdichte $j$ konstant ist.  Wenn wir die im Kern abfallende magnetische Spannung vernachlässigen gilt wegen $\Theta = R\Phi$ und des magnetische Widerstands im Luftspalt $R = \frac{d}{\mu_0A}$ dass    

$$ 
4d(l_z-d)j = \Theta = \frac{d}{\mu_0A}\Phi = \frac{d}{\mu_0}B.
$$

Die Stromdichte ergibt sich also ungefähr als 
$$
 j \approx \frac{B}{4\mu_0(l_z-d) } \approx 5*10^5.
$$

Es stellt sich heraus, dass $j \approx 7*10^5$ zum gewünschten einen Tesla führt. 

In [ ]:
fes = HCurl(mesh, order = 3, dirichlet="outer", nograds=True)
u,v = fes.TnT()
from math import pi


mu0 = 4*pi*1e-7
mur = { "core" : 1e5, "coil" : 1, "air" : 1, "plate": 1}
nu = CoefficientFunction([1/(mu0*mur[mat]) for mat in mesh.GetMaterials()]) # max: 10^2, min:10^6 
reg = 1e-6*nu

a = BilinearForm(fes)
a += nu*curl(u)*curl(v)*dx + reg*u*v*dx
c = Preconditioner(a, "bddc")

f = LinearForm(fes)
f += CoefficientFunction((-y,lx-d/2+x,0))*CoefficientFunction(1/sqrt(y**2+(lx-d/2+x)**2)*7*1e5)*v*dx(mesh.Materials('coil'))

gfu = GridFunction(fes)
# Was passiert wenn blf schon linear ist?
# test = gfu.vec.CreateVector()
# test[:] = 0

with TaskManager():
    a.Assemble()
#     a.AssembleLinearization(test)
    f.Assemble()
    solvers.CG(sol=gfu.vec, rhs=f.vec, mat=a.mat, pre=c.mat)


# a.AssembleLinearization(gfu.vec)
# solvers.CG(sol=gfu.vec, rhs=f.vec, mat=a.mat, pre=c.mat)
def show():
    Draw (curl(gfu), mesh, "B-field", draw_surf=False)
    Draw (nu*curl(gfu), mesh, "H-field", draw_surf=False)
    mip = mesh(lx-d/2, 0, 0)
    print(curl(gfu)(mip), np.linalg.norm(curl(gfu)(mip)))
# Draw(CoefficientFunction((-y,lx-d/2+x,0))*CoefficientFunction(1/sqrt(y**2+(lx-d/2+x)**2)), mesh)
show()

### Magnetostatisches Problem mit Stahlplatte

Ohne dem Regularisierungsterm lautet das nichtlineare Problem:


Finde $ u \in H(\Curl)$ sodass 
$$
\int \underbrace{\frac{1}{\mu(|B|)}\Curl u}_{=H} \Curl v = \int j v \qquad  \forall \, v \in H(\Curl) \tag{1}
$$

Wenn $u$ das Mimimierungsproblem 
$$
F(u) := \int \omega(|\Curl u|) - \int ju  \qquad \min !
$$
mit 
$$
w(x) = \int_0^{x} \frac{1}{\mu(s)}s ~ ds
$$
löst, dann erfüllt $u$ auch (1). 


Um das einzusehen berechne die erste Variation von $F$:
$$
\begin{split}
\lim_{\epsilon \rightarrow 0} \frac{d}{d\epsilon} F(u+\epsilon v) &= \int \lim_{\epsilon \rightarrow 0}\frac{1}{\mu(|\Curl(u+\epsilon v)|)}|\Curl(u+\epsilon v)|\frac{\Curl(u+\epsilon v)}{|\Curl(u+\epsilon v)|}\Curl v  - \int jv\\
&= \int \frac{1}{\mu(|\Curl u|)}\Curl u \Curl v  - \int jv
\end{split}
$$

Die folgende H-B Kurve ist aus der TEAM 13 Lösung von Valentin Hanser:

In [ ]:
from matplotlib import pyplot as plt

H_KL = [ -4.47197834e-13, 1.60000000e+01, 3.00000000e+01, 5.40000000e+01\
, 9.30000000e+01, 1.43000000e+02, 1.91000000e+02, 2.10000000e+02 \
, 2.22000000e+02, 2.33000000e+02, 2.47000000e+02, 2.58000000e+02\
, 2.72000000e+02, 2.89000000e+02, 3.13000000e+02, 3.42000000e+02\
, 3.77000000e+02, 4.33000000e+02, 5.09000000e+02, 6.48000000e+02\
, 9.33000000e+02, 1.22800000e+03, 1.93400000e+03, 2.91300000e+03\
, 4.99300000e+03, 7.18900000e+03, 9.42300000e+03, 9.42300000e+03\
, 1.28203768e+04, 1.65447489e+04, 2.07163957e+04, 2.55500961e+04\
, 3.15206135e+04, 4.03204637e+04, 7.73038295e+04, 1.29272791e+05\
, 1.81241752e+05, 2.33210713e+05, 2.85179674e+05, 3.37148635e+05\
, 3.89117596e+05, 4.41086557e+05, 4.93055518e+05, 5.45024479e+05\
, 5.96993440e+05, 6.48962401e+05, 7.00931362e+05, 7.52900323e+05\
, 8.04869284e+05, 8.56838245e+05, 9.08807206e+05, 9.60776167e+05\
, 1.01274513e+06, 1.06471409e+06, 1.11668305e+06, 1.16865201e+06\
, 1.22062097e+06, 1.27258993e+06, 1.32455889e+06, 1.37652785e+06\
, 1.42849682e+06, 1.48046578e+06, 1.53243474e+06, 1.58440370e+06\
, 1.63637266e+06, 1.68834162e+06, 1.74031058e+06, 1.79227954e+06\
, 1.84424850e+06, 1.89621746e+06, 1.94818643e+06, 2.00015539e+06\
, 2.05212435e+06, 2.10409331e+06, 2.15606227e+06, 2.20803123e+06\
, 2.26000019e+06]

B_KL = [ 0.00000000e+00, 2.50000000e-03, 5.00000000e-03, 1.25000000e-02\
, 2.50000000e-02, 5.00000000e-02, 1.00000000e-01, 2.00000000e-01\
, 3.00000000e-01, 4.00000000e-01, 5.00000000e-01, 6.00000000e-01\
, 7.00000000e-01, 8.00000000e-01, 9.00000000e-01, 1.00000000e+00\
, 1.10000000e+00, 1.20000000e+00, 1.30000000e+00, 1.40000000e+00\
, 1.50000000e+00, 1.55000000e+00, 1.60000000e+00, 1.65000000e+00\
, 1.70000000e+00, 1.75000000e+00, 1.80000000e+00, 1.80000000e+00\
, 1.86530612e+00, 1.93061224e+00, 1.99591837e+00, 2.06122449e+00\
, 2.12653061e+00, 2.19183673e+00, 2.25714286e+00, 2.32244898e+00\
, 2.38775510e+00, 2.45306122e+00, 2.51836735e+00, 2.58367347e+00\
, 2.64897959e+00, 2.71428571e+00, 2.77959184e+00, 2.84489796e+00\
, 2.91020408e+00, 2.97551020e+00, 3.04081633e+00, 3.10612245e+00\
, 3.17142857e+00, 3.23673469e+00, 3.30204082e+00, 3.36734694e+00\
, 3.43265306e+00, 3.49795918e+00, 3.56326531e+00, 3.62857143e+00\
, 3.69387755e+00, 3.75918367e+00, 3.82448980e+00, 3.88979592e+00\
, 3.95510204e+00, 4.02040816e+00, 4.08571429e+00, 4.15102041e+00\
, 4.21632653e+00, 4.28163265e+00, 4.34693878e+00, 4.41224490e+00\
, 4.47755102e+00, 4.54285714e+00, 4.60816327e+00, 4.67346939e+00\
, 4.73877551e+00, 4.80408163e+00, 4.86938776e+00, 4.93469388e+00\
, 5.00000000e+00]
bh_curve = BSpline (2, [0]+list(B_KL), list(H_KL)) # [0] + is needed!
energy_dens = bh_curve.Integrate()

plt.figure(1, figsize=[12, 9])
plt.clf()
plt.plot(H_KL, B_KL, '.-r')
plt.xlim(0, 2000)
plt.ylim(0, 2.12)
plt.grid()
plt.xlabel("Magnetic Field Strength H in A/m")
plt.ylabel("Magnetic Flux Density B in T")

### Implementierung ähnlich wie im Beipiel

In dem Beipiel wird die schwache Formulierung mit dem Minimierungsproblem gemischt. Gibt es dafür einen Grund?

In [ ]:
fes = HCurl(mesh, order = 3, dirichlet="outer", nograds=True)
u,v = fes.TnT()
from math import pi

j = 3*1e5
mu0 = 4*pi*1e-7
mur = { "core" : 1e8, "coil" : 1, "air" : 1}

a = BilinearForm(fes)
for mat in mur.keys():
    a += 1/(mu0*mur[mat])*curl(u)*curl(v)*dx(mat) + 1e-6*1/(mu0*mur[mat])*u*v*dx(mat)
# a += Variation(energy_dens(sqrt(1e-12+ curl(u)*curl(u))) * dx('plate')) # 1e-12+ ?
a += SymbolicEnergy(energy_dens(sqrt(1e-12+ curl(u)*curl(u))), definedon = 'plate') # 1e-12+ ?
a += 1e-3*u*v*dx('plate')
    
## TODO: hier preconditioning Beispiel
c = Preconditioner(a, "bddc")

f = LinearForm(fes)
f += CoefficientFunction((-y,lx-d/2+x,0))*CoefficientFunction(1/sqrt(y**2+(lx-d/2+x)**2)*j)*v*dx(mesh.Materials('coil'))


TOL = 1e-10
MAX_IT = 60

with TaskManager():

    f.Assemble()

    gfu = GridFunction(fes)
    res = gfu.vec.CreateVector()
    w = gfu.vec.CreateVector()
    tmp = gfu.vec.CreateVector()
    
    gfu.vec[:] = 0

    err = 1
    it = 0
    
    while err > TOL and it < MAX_IT:
        it += 1
        
        e_old = a.Energy(gfu.vec) - InnerProduct(f.vec, gfu.vec)
        
        a.Apply(gfu.vec, tmp)
        res.data = tmp - f.vec
        
        a.AssembleLinearization(gfu.vec)
        solvers.CG(sol=w, rhs=res, mat=a.mat, pre=c.mat)
        
        err = abs(InnerProduct(w, res))
        
        print("\n error {} in iteration {}".format(err, it))
        
        tmp.data = gfu.vec - w
        
        e_new = a.Energy(tmp) - InnerProduct(f.vec, tmp)
        print("old energy: {}, new energy: {}".format(e_old, e_new))
        
        tau = 1
        cnt = 1
        while e_new > e_old:
            tau *= max(0.1, 1-0.1*cnt)
            cnt += 1
            tmp.data = gfu.vec - tau*w
            e_new = a.Energy(tmp) - InnerProduct(f.vec, tmp)
            print("new energy: {} with tau: {}".format( e_new, tau))
        
        gfu.vec.data = tmp


def show():
    Draw (curl(gfu), mesh, "B-field", draw_surf=False)
    mip = mesh(lx-d/2, 0, 0)
    print(curl(gfu)(mip), np.linalg.norm(curl(gfu)(mip)))

In [ ]:
show()

### Ganz als Minimierungsproblem

Minimiere das Funktional 

$$
F(u) := \int_{\text{plate}} \omega(|\Curl u|) + \int_{\Omega\setminus\text{plate}} \frac{1}{2\mu}\Curl u + \reg \int \frac{1}{2} u  - \int ju 
$$

Funktioniert aus irgendeinem Grund viel besser.

In [ ]:
fes = HCurl(mesh, order = 3, dirichlet="outer", nograds=True)
u = fes.TrialFunction()
from math import pi

j = 5*1e5
mu0 = 4*pi*1e-7
mur = { "core" : 1e10, "coil" : 1, "air" : 1}

a = BilinearForm(fes)
for mat in mur.keys():
    a += Variation( 1/(2*mu0*mur[mat])*curl(u)*curl(u)*dx(mat) + 1e-6*1/(2*mu0*mur[mat])*u*u*dx(mat))
a += Variation(energy_dens(sqrt(1e-12+ curl(u)*curl(u))) * dx('plate')) # 1e-12+ ?
# a += SymbolicEnergy(energy_dens(sqrt(1e-12+ curl(u)*curl(u))), definedon = 'plate') # 1e-12+ ?
a += Variation(1e-3/2*u*u*dx('plate') )
a += Variation(CoefficientFunction((-y,lx-d/2+x,0))*CoefficientFunction(-1/sqrt(y**2+(lx-d/2+x)**2)*j)*u*dx('coil'))
    
## TODO: hier preconditioning Beispiel
c = Preconditioner(a, "bddc")

# f = LinearForm(fes)
# f += CoefficientFunction((-y,lx-d/2+x,0))*CoefficientFunction(1/sqrt(y**2+(lx-d/2+x)**2)*j)*v*dx(mesh.Materials('coil'))


TOL = 1e-10
MAX_IT = 60

with TaskManager():

    gfu = GridFunction(fes)
    res = gfu.vec.CreateVector()
    w = gfu.vec.CreateVector()
    tmp = gfu.vec.CreateVector()
    
    gfu.vec[:] = 0

    err = 1
    it = 0
    
    while err > TOL and it < MAX_IT:
        it += 1
        
        e_old = a.Energy(gfu.vec)
        
        a.Apply(gfu.vec, res)
        
        a.AssembleLinearization(gfu.vec)
        solvers.CG(sol=w, rhs=res, mat=a.mat, pre=c.mat)
        
        err = abs(InnerProduct(w, res))
        
        print("\n error {} in iteration {}".format(err, it))
        
        tmp.data = gfu.vec - w
        
        e_new = a.Energy(tmp)
        print("old energy: {}, new energy: {}".format(e_old, e_new))
        
        tau = 1
        cnt = 1
        while e_new > e_old:
            tau *= max(0.1, 1-0.1*cnt)
            cnt += 1
            tmp.data = gfu.vec - tau*w
            e_new = a.Energy(tmp)
            print("new energy: {} with tau: {}".format( e_new, tau))
        
        gfu.vec.data = tmp


def show():
    Draw (curl(gfu), mesh, "B-field", draw_surf=False)
    mip = mesh(lx-d/2, 0, 0)
    print(curl(gfu)(mip), np.linalg.norm(curl(gfu)(mip)))

In [ ]:
show()